In [1]:
# import required libriaries
from pathlib import Path
from helpers import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [4]:
df_train.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
82179,Yoshinori_Muto,25,4,0,Newcastle United,Norwich,0.637862,0.300208,True,0,...,0,0,0.0,-139,21,160,0,2020-02-01T15:00:00Z,1920,0.0
82180,Youri_Tielemans,25,3,79,Leicester City,Chelsea,1.035290,1.745380,True,9,...,2,2,19.0,-18539,2505,21044,0,2020-02-01T12:30:00Z,1920,1.0
82181,Yves_Bissouma,25,3,0,Brighton and Hove Albion,West Ham United,0.462137,0.740968,False,0,...,3,3,0.0,-32,28,60,0,2020-02-01T15:00:00Z,1920,1.0
82182,Çaglar_Söyüncü,25,2,90,Leicester City,Chelsea,1.035290,1.745380,True,1,...,2,2,0.0,-94590,25071,119661,0,2020-02-01T12:30:00Z,1920,1.0
82183,Ørjan_Nyland,25,1,0,Aston Villa,Bournemouth,0.539900,0.707921,False,0,...,1,2,0.0,-792,645,1437,0,2020-02-01T15:00:00Z,1920,1.0


In [5]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

26

In [6]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [9]:
df_latest
#df_latest[df_latest['player'] == 'Kevin_De Bruyne']

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Chris_Mepham,26,2,0,Bournemouth,Sheffield United,0.704973,0.293816,False,0,...,49176,1,2,0.0,-1083,30,1113,0,2020-02-09T14:00:00Z,1920
1,Paulo_Gazzaniga,26,1,0,Tottenham Hotspur,Aston Villa,1.845990,0.537651,False,0,...,193598,3,2,0.0,-43615,2389,46004,0,2020-02-16T14:00:00Z,1920
2,John_Egan,26,2,90,Sheffield United,Bournemouth,0.293816,0.704973,True,6,...,135290,1,2,54.0,38911,43550,4639,0,2020-02-09T14:00:00Z,1920
3,Max_Meyer,26,3,22,Crystal Palace,Everton,0.473067,0.978426,False,1,...,5850,1,3,0.0,-15,28,43,0,2020-02-08T12:30:00Z,1920
4,Fabian_Delph,26,2,0,Everton,Crystal Palace,0.978426,0.473067,True,0,...,13515,1,3,0.0,-1022,34,1056,0,2020-02-08T12:30:00Z,1920
5,James_Justin,26,2,0,Leicester City,Wolverhampton Wanderers,1.030978,0.708675,False,0,...,7369,0,0,0.0,-64,31,95,0,2020-02-14T20:00:00Z,1920
6,Che_Adams,26,4,4,Southampton,Burnley,0.452499,0.402621,True,1,...,23519,2,1,0.0,-56,318,374,0,2020-02-15T12:30:00Z,1920
7,Mahmoud Ahmed_Ibrahim Hassan,26,3,20,Aston Villa,Tottenham Hotspur,0.537651,1.845990,True,1,...,84772,3,2,12.0,-1291,1945,3236,0,2020-02-16T14:00:00Z,1920
8,Tyreece_John-Jules,26,4,0,Arsenal,Newcastle United,1.437198,0.640348,True,0,...,360,0,4,0.0,-15,50,65,0,2020-02-16T16:30:00Z,1920
9,Adrian_Mariappa,26,2,90,Watford,Brighton and Hove Albion,0.476769,0.467411,False,-1,...,28311,1,1,0.0,911,2584,1673,1,2020-02-08T17:30:00Z,1920


In [10]:
df_latest.shape

(623, 34)

In [11]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [12]:
last_play_proba.head()

,player,play_proba
0,Shkodran_Mustafi,1.0
1,Héctor_Bellerín,1.0
2,Sead_Kolasinac,0.5
3,Ainsley_Maitland-Niles,1.0
4,Sokratis_Papastathopoulos,1.0


In [13]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [14]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
0,Chris_Mepham,26,2,0,Bournemouth,Sheffield United,0.704973,0.293816,False,0,...,1,2,0.0,-1083,30,1113,0,2020-02-09T14:00:00Z,1920,0.0
1,Paulo_Gazzaniga,26,1,0,Tottenham Hotspur,Aston Villa,1.845990,0.537651,False,0,...,3,2,0.0,-43615,2389,46004,0,2020-02-16T14:00:00Z,1920,1.0
2,John_Egan,26,2,90,Sheffield United,Bournemouth,0.293816,0.704973,True,6,...,1,2,54.0,38911,43550,4639,0,2020-02-09T14:00:00Z,1920,1.0
3,Max_Meyer,26,3,22,Crystal Palace,Everton,0.473067,0.978426,False,1,...,1,3,0.0,-15,28,43,0,2020-02-08T12:30:00Z,1920,1.0
4,Fabian_Delph,26,2,0,Everton,Crystal Palace,0.978426,0.473067,True,0,...,1,3,0.0,-1022,34,1056,0,2020-02-08T12:30:00Z,1920,0.0
5,James_Justin,26,2,0,Leicester City,Wolverhampton Wanderers,1.030978,0.708675,False,0,...,0,0,0.0,-64,31,95,0,2020-02-14T20:00:00Z,1920,1.0
6,Che_Adams,26,4,4,Southampton,Burnley,0.452499,0.402621,True,1,...,2,1,0.0,-56,318,374,0,2020-02-15T12:30:00Z,1920,1.0
7,Mahmoud Ahmed_Ibrahim Hassan,26,3,20,Aston Villa,Tottenham Hotspur,0.537651,1.845990,True,1,...,3,2,12.0,-1291,1945,3236,0,2020-02-16T14:00:00Z,1920,1.0
8,Tyreece_John-Jules,26,4,0,Arsenal,Newcastle United,1.437198,0.640348,True,0,...,0,4,0.0,-15,50,65,0,2020-02-16T16:30:00Z,1920,0.0
9,Adrian_Mariappa,26,2,90,Watford,Brighton and Hove Albion,0.476769,0.467411,False,-1,...,1,1,0.0,911,2584,1673,1,2020-02-08T17:30:00Z,1920,1.0


In [15]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [16]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
82802,Heurelho da Silva_Gomes,26,1,0,Watford,Brighton and Hove Albion,0.476769,0.467411,False,0,...,1,1,0.0,-60,112,172,0,2020-02-08T17:30:00Z,1920,1.0
82803,Joseph_Anang,26,1,0,West Ham United,Manchester City,0.758450,2.653289,False,0,...,0,2,0.0,132,529,397,0,2020-02-19T19:30:00Z,1920,1.0
82804,Erik_Pieters,26,2,0,Burnley,Southampton,0.402621,0.452499,False,0,...,2,1,0.0,-8007,617,8624,0,2020-02-15T12:30:00Z,1920,1.0
82805,Japhet_Tanganga,26,2,0,Tottenham Hotspur,Aston Villa,1.845990,0.537651,False,0,...,3,2,0.0,68650,74632,5982,0,2020-02-16T14:00:00Z,1920,1.0
82806,Ravel_Morrison,26,3,0,Sheffield United,Bournemouth,0.293816,0.704973,True,0,...,1,2,0.0,-322,11,333,0,2020-02-09T14:00:00Z,1920,0.0


In [17]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train.csv')

In [18]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [19]:
fixtures.head(10)

,home_team,away_team,gw
260,Arsenal,Everton,27
261,Burnley,Bournemouth,27
262,Chelsea,Tottenham Hotspur,27
263,Crystal Palace,Newcastle United,27
264,Leicester City,Manchester City,27
265,Liverpool,West Ham United,27
266,Manchester United,Watford,27
267,Sheffield United,Brighton and Hove Albion,27
268,Southampton,Aston Villa,27
269,Wolverhampton Wanderers,Norwich,27


In [20]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [21]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,27,Arsenal,Everton,3,11
1,27,Burnley,Bournemouth,90,91
2,27,Chelsea,Tottenham Hotspur,8,6
3,27,Crystal Palace,Newcastle United,31,4
4,27,Leicester City,Manchester City,13,43


In [22]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [23]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [24]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,27,Everton,Arsenal,Alex_Iwobi,3,56,100,False
1,27,Everton,Arsenal,Lucas_Digne,2,58,100,False
2,27,Everton,Arsenal,Yerry_Mina,2,53,100,False
3,27,Everton,Arsenal,Michael_Keane,2,52,100,False
4,27,Everton,Arsenal,Seamus_Coleman,2,53,100,False


In [25]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,27,Arsenal,Everton,Shkodran_Mustafi,2,51,100,True
1,27,Arsenal,Everton,Héctor_Bellerín,2,54,100,True
2,27,Arsenal,Everton,Sead_Kolasinac,2,52,100,True
3,27,Arsenal,Everton,Ainsley_Maitland-Niles,2,46,100,True
4,27,Arsenal,Everton,Sokratis_Papastathopoulos,2,49,75,True


In [26]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 19.4,
 'Aaron_Cresswell': 90.0,
 'Aaron_Lennon': 0.25,
 'Aaron_Mooy': 62.75,
 'Aaron_Ramsdale': 67.5,
 'Aaron_Wan-Bissaka': 90.0,
 'Abd-Al-Ali Morakinyo Olaposi_Koiki': 0.0,
 'Abdoulaye_Doucouré': 89.0,
 'Adam_Idah': 12.8,
 'Adam_Lallana': 7.571428571428571,
 'Adam_Masina': 90.0,
 'Adam_Smith': 90.0,
 'Adam_Webster': 72.0,
 'Adama_Traoré': 86.25,
 'Addji Keaninkin Marc-Israel_Guehi': 0.0,
 'Adrian_Mariappa': 90.0,
 'Adrián_San Miguel del Castillo': 0.0,
 'Ahmed_El Mohamady': 18.0,
 'Ainsley_Maitland-Niles': 36.0,
 'Akin_Famewo': 0.0,
 'Albian_Ajeti': 0.75,
 'Alex_Iwobi': 32.0,
 'Alex_McCarthy': 90.0,
 'Alex_Oxlade-Chamberlain': 71.71428571428571,
 'Alexander_Tettey': 85.8,
 'Alexandre_Lacazette': 69.2,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 27.4,
 'Alisson_Ramses Becker': 90.0,
 'Allan_Saint-Maximin': 56.0,
 'Andre_Gray': 2.0,
 'Andreas_Christensen': 81.0,
 'Andreas_Pereira': 53.8,
 'Andrew_Robertson': 90.0,
 'Andrew_Surman': 17.4,
 'Andros_Townsend': 6.5,
 

In [27]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [28]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,27,Arsenal,Everton,Shkodran_Mustafi,2,5.1,1.00,True,1.437198,0.978426,1920,67.500
1,27,Arsenal,Everton,Héctor_Bellerín,2,5.4,1.00,True,1.437198,0.978426,1920,90.000
2,27,Arsenal,Everton,Sead_Kolasinac,2,5.2,1.00,True,1.437198,0.978426,1920,45.000
3,27,Arsenal,Everton,Ainsley_Maitland-Niles,2,4.6,1.00,True,1.437198,0.978426,1920,36.000
4,27,Arsenal,Everton,Sokratis_Papastathopoulos,2,4.9,0.75,True,1.437198,0.978426,1920,16.875
5,27,Arsenal,Everton,Nacho_Monreal,2,5.0,0.00,True,1.437198,0.978426,1920,0.000
6,27,Arsenal,Everton,Laurent_Koscielny,2,5.0,0.00,True,1.437198,0.978426,1920,0.000
7,27,Arsenal,Everton,Konstantinos_Mavropanos,2,4.4,0.00,True,1.437198,0.978426,1920,0.000
8,27,Arsenal,Everton,Carl_Jenkinson,2,4.5,0.00,True,1.437198,0.978426,1920,0.000
9,27,Arsenal,Everton,Rob_Holding,2,4.5,1.00,True,1.437198,0.978426,1920,1.800


In [29]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')